# Jupyter widgets
[Jupyter widgets](https://ipywidgets.readthedocs.io/en/latest/index.html) are visual components that enhance the interaction with a Jupyter notebook.

In [1]:
from IPython.display import display
from ipywidgets import Checkbox, VBox, HBox, Layout, Text, Image
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [2]:
a = widgets.Checkbox(description='a', layout=Layout(object_position='left'))
b = widgets.Checkbox(description='b', object_position='left')
c = widgets.Checkbox(description='c')

In [3]:
def f(a, b, c):
    print('{}, {}, {}'.format(a, b, c))

In [6]:
out = widgets.interactive_output(f, {'a': a, 'b': b, 'c': c})
widgets.HBox([widgets.HBox([a, b, c]), out])